# Notebook for ML deployment of di-f Correlation experiments

## Experiment name: mxretailsalary1

## Experiment General Data
### Team roles:
* **PipeMaster**: jag.pascoe
* **BizEngineer**: 
* **DataEngineer**:
* **MLEngineer**:
* **SWEngineer**:

### Description (Use case):
Predict salary per day estimation to be obtained for working in retail sector in any state of Mexico.
Supposing you are looking for being hired in a Retail Business in any of Mexico's state you want to. You want to predict which would be the base salary per day you might get as attendant of that retail business. This salary not include any commision, tax, or any other concept.

### Type of experiment: Correlation
### Independent Variables (inputs):
1) State of Mexico where you are supposing to get hired (CAT). 
2) How many employees (including yourself) work in that particular business now (NUMBER)
3) How much sales per day in average, you estimate you will provide to that business in pesos (FLOAT)

### Dependent Variables (outputs):
1) Estimated base salary per day (FLOAT)

## Experiment preparation, imports and config.yaml

In [9]:
%load_ext autoreload
%autoreload 2
# The %load_ext autoreload and %autoreload 2 magic commands are used to automatically 
# reload modules when they are changed. This can be useful when you are developing code 
# in an interactive environment, as it allows you to see the changes you make to your modules 
# without having to restart the kernel.
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import pandas as pd
import numpy as np
import os


# for global initialization: NOT RECOMMENDED
#initialize(version_base=None, config_path="../src/conf")
#compose(config_name='config')

with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name='config')
    print(cfg)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'general_ml': {'seed': 123, 'encoding': 'iso-8859-1', 'cloud': 'AWS'}, 'paths': {'project_dir': '/home/jagpascoe/democlient-sklearn/dif-s-mxretailsalary1', 'raw_data_dir': '${paths.project_dir}/data/raw', 'processed_data_dir': '${paths.project_dir}/data/processed', 'interim_data_dir': '${paths.project_dir}/data/interim', 'reports_dir': '${paths.project_dir}/reports', 'models_dir': '${paths.project_dir}/models', 'api_pycaret_dir': '${paths.project_dir}/API/API_Pycaret', 'api_fastapi_dir': '${paths.project_dir}/API/API_FastAPI', 'api_flask_dir': '${paths.project_dir}/API/API_Flask'}, 'file_names': {'raw_file': 'raw-data.csv', 'processed_data': 'processed_data.csv', 'processed_unseen_data': 'processed_unseen_data.csv', 'data_file': 'datafile.csv', 'train_features': 'train_features.csv', 'train_labels': 'train_labels.csv', 'validation_features': 'valid_features.csv', 'validation_labels': 'valid_labels.

In [10]:
#Choose the Ml model to be applied, among: regression, Classifications, time_series, Clustering, NLP
from pycaret.regression import *
from pycaret import version_
version_

'3.0.3'

## Loading the model and unseen_data

### first the model
This function loads a previously saved pipeline.

In [11]:
selected_model = load_model(os.path.join(cfg.paths.models_dir, cfg.file_names.ml_profiling_best))
selected_model

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['income_employee_day',
                                             'employees_business'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['state'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['state'],
                                    t...
                ('actual_estimator',
                 VotingRegressor(estimators=[('CatBoost Regressor',
                                              <catboost.core.CatBoostRegressor object at 0x7f5b8e6a5330>),
                                             ('Gradient Boosting Regressor',
                                              GradientBoostingRegressor(random_state=123)),
                                             ('Huber Regressor',
                                              HuberRegressor()),
                                             ('Bayesian Ridge',
                                              BayesianRidge()),
                                             ('Ridge Regression',
                                              Ridge(random_state=123))],
                                 n_jobs=-1,
                                 weights=[0.4, 0.3, 0.2, 0.07, 0.03]))])

### then, unseen_data

In [12]:
unseen_data = pd.read_csv(os.path.join(cfg.paths.processed_data_dir, cfg.file_names.processed_unseen_data), 
                   #encoding=cfg.general_ml.encoding,
                   )
unseen_data.head()

,state,income_employee_day,employees_business,salary_employee_day
0,Oaxaca,6021.068300,4,395.386549
1,Veracruz,12113.568050,6,573.941444
2,Michoacan,28979.260706,4,291.728084
3,Puebla,10824.636591,5,435.024135
4,Yucatan,32134.630792,11,881.853456


### now predict on unseen_data
This function generates the label using a trained model.  When data is None, it predicts label and score on the holdout set. 

In [8]:
predict_model(selected_model, data=unseen_data) 

,state,income_employee_day,employees_business,salary_employee_day,prediction_label
0,Oaxaca,6021.068359,4,395.386536,192.142239
1,Veracruz,12113.568359,6,573.941467,451.583270
2,Michoacan,28979.259766,4,291.728088,374.386688
3,Puebla,10824.636719,5,435.024139,383.076618
4,Yucatan,32134.630859,11,881.853455,725.390608
...,...,...,...,...,...
1245,Oaxaca,8606.669922,6,331.792908,314.955454
1246,Yucatan,8553.745117,7,220.410995,259.652198
1247,Durango,45161.300781,12,1069.434692,906.899124
1248,Nayarit,23687.837891,10,645.820679,642.583502


## Deployment

### as API with FastAPI
This function takes an input model and creates a POST API for inference. It only creates the API and doesn't run it automatically. 
Once you initialize API with the !python command. You can see the server on localhost:8000/docs.

In [14]:
load_experiment(os.path.join(cfg.paths.models_dir, "experiment-"+cfg.file_names.ml_profiling_best),
                data = pd.read_csv(os.path.join(cfg.paths.processed_data_dir, cfg.file_names.processed_data), 
                   #encoding=cfg.general_ml.encoding,
                   ))

,Description,Value
0,Session id,123
1,Target,salary_employee_day
2,Target type,Regression
3,Original data shape,"(1250, 4)"
4,Transformed data shape,"(1250, 35)"
5,Transformed train set shape,"(875, 35)"
6,Transformed test set shape,"(375, 35)"
7,Numeric features,2
8,Categorical features,1
9,Preprocess,True


In [16]:
create_api(selected_model, os.path.join(cfg.paths.models_dir, cfg.file_names.ml_profiling_best))

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python /home/jagpascoe/democlient-sklearn/dif-s-mxretailsalary1/models/ml_profiling_best.py


## creating docker config files
This function creates a Dockerfile and requirements.txt for productionalizing API end-point.

In [16]:

create_docker(os.path.join(cfg.paths.api, cfg.file_names.api_ml_profiling))

Writing requirements.txt
Writing Dockerfile
Dockerfile and requirements.txt successfully created.
    To build image you have to run --> !docker image build -f "Dockerfile" -t IMAGE_NAME:IMAGE_TAG .
            


## create an app
This function creates a basic gradio app for inference. It will later be expanded for other app types such Streamlit.

In [17]:
create_app(selected_model)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
